<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" style="height:450px;" width=500/></p>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>
<h3 style="text-align: center;"><b>Продвинутый поток (часть 2). Весна 2021</b></h3>

<h1 style="text-align: center;"><b>Language modeling.</b></h1>

Для начала загрузим датасет, состоящий из сэмплов кода на языке Python. Датасет представлен гитхабом. [Про датасет](https://github.blog/2019-09-26-introducing-the-codesearchnet-challenge/).

Для препроцессинга будем использовать уже известную нам библиотеку `datasets` от Huggingface.

In [1]:
!pip install -q datasets

     |████████████████████████████████| 290 kB 4.2 MB/s 
     |████████████████████████████████| 243 kB 38.4 MB/s 
     |████████████████████████████████| 132 kB 46.3 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 1.1 MB 22.0 MB/s 
     |████████████████████████████████| 192 kB 43.8 MB/s 
     |████████████████████████████████| 160 kB 50.1 MB/s 
     |████████████████████████████████| 271 kB 45.9 MB/s 


In [2]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!unzip -p python.zip python/final/jsonl/train/python_train_0.jsonl.gz > train.jsonl.gz
!unzip -p python.zip python/final/jsonl/test/python_test_0.jsonl.gz > test.jsonl.gz

--2021-11-08 15:17:07--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.169.72
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.169.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  74.1MB/s    in 12s     

2021-11-08 15:17:20 (72.5 MB/s) - ‘python.zip’ saved [940909997/940909997]



In [3]:
# decompress this gzip file
!gzip -d train.jsonl.gz
!gzip -d test.jsonl.gz

Загружать датасеты можно не только из хаба, но и из диска. Для этого достаточно указать формат и путь до файла.

In [4]:
from datasets import load_dataset  
dataset = load_dataset(
    "json",
    data_files=[
        "train.jsonl",
    ],
)

Using custom data configuration default-6e65f87f167045df


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6e65f87f167045df/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 30000
    })
})

Ограничим число уникальных слов до `40000`.

In [6]:
from collections import Counter


vocab_size = 40000
stats = Counter()

for item in dataset["train"]:
    stats.update(item["code_tokens"])
tokens = dict(stats.most_common(vocab_size)).keys()

Посмотрим на самые часто встречающиеся слова

In [7]:
stats.most_common(20)

[('(', 299562),
 (')', 299456),
 ('.', 283659),
 (',', 267888),
 ('=', 238494),
 (':', 176959),
 ('[', 99465),
 (']', 99448),
 ('self', 71449),
 ('if', 65763),
 ('return', 40852),
 ('None', 37311),
 ('def', 32278),
 ('in', 29951),
 ('*', 21773),
 ('not', 21291),
 ('name', 21205),
 ('ret', 20862),
 ('1', 19494),
 ('for', 19364)]

Добавим служебные токены

In [8]:
PAD = 0
UNK = 1
EOS = 2

token2idx = {"[PAD]": 0, "[UNK]": 1, "[EOS]": 2}

for idx, token in enumerate(tokens):
    token2idx[token] = idx + 3

Переведем токены в индексы

In [9]:
def encode(token):
    if token in token2idx.keys():
        return token2idx[token]
    return UNK

In [10]:
dataset = dataset.map(
    lambda item: {
        "features": [encode(token) for token in item["code_tokens"]] + [EOS]
    }
)

  0%|          | 0/30000 [00:00<?, ?ex/s]

## N-gram

 Наченм с простейшей модели. Она основывается на статистическом методе. Итак, в языковом моделировании мы хотим максимизировать вероятность нашего текста по мнению модели, то есть:
 $$
\mathrm{P}(\mathrm{W})=\mathrm{P}\left(\mathrm{w}_{1}, \mathrm{w}_{2}, \mathrm{w}_{3}, \mathrm{w}_{4}, \mathrm{w}_{5} \ldots \mathrm{w}_{\mathrm{n}}\right)
$$


Вспомним, что можно переписать:

$$
P\left(x_{1}, x_{2}, x_{3}, \ldots, x_{n}\right)=P\left(x_{1}\right) P\left(x_{2} \mid x_{1}\right) P\left(x_{3} \mid x_{1}, x_{2}\right) \ldots P\left(x_{n} \mid x_{1}, \ldots, x_{n-1}\right)
$$

Тогда:

$$
P\left(w_{1} w_{2} \ldots w_{n}\right)=\prod_{i} P\left(w_{i} \mid w_{1} w_{2} \ldots w_{i-1}\right)
$$

Однако число вероятностей вида $P\left(w_{i} \mid w_{1} w_{2} \ldots w_{i-1}\right)$ растет очень быстро. Поэтому используют некоторое предположение которое называется **марковковское приближение**. Формулируется оно так:

$$
P\left(w_{1} w_{2} \ldots w_{n}\right) \approx \prod_{i} P\left(w_{i} \mid w_{i-k} \ldots w_{i-1}\right)
$$

То есть мы считаем, что текущее слово зависит только от $k$ предыдущих.

$$
P\left(w_{i} \mid w_{1} w_{2} \ldots w_{i-1}\right) \approx P\left(w_{i} \mid w_{i-k} \ldots w_{i-1}\right)
$$


In [11]:
import numpy as np
from collections import Counter, defaultdict

from tqdm.notebook import tqdm


class NGramModel(object):
    """
    Структура этой реализации n-граммной модели следующая:
    self.ngrams – словарь, который на каждый (token_0, ..., token_(n-1)) – n-1 tuple из токенов
        хранит частоту появления следующего токена. Для подсчета числа токенов воспользуемся
        Counter
    self.tokenize_func – функция токенизации текста. С её помощью будем получать токены.
    """
    def __init__(self, n=2):
        self.ngrams = defaultdict(Counter)
        self.n = n
        self.tokenize_func = None
        
    def compute_ngrams(self, dataset):
        self.ngrams = defaultdict(Counter)
        for row in tqdm(dataset):
            ngram = [PAD] * self.n
            for token in row["features"]:
                ngram[:-1] = ngram[1:]
                ngram[-1] = token
                self.ngrams[tuple(ngram[:-1])].update([ngram[-1]])
            
    def get_log_probs(self, prefix, min_log_pr=-15):
        """
        Функция, которая будет возвращать логарифмы частот появления токенов
        """
        if len(prefix) < self.n - 1:
            prefix = [PAD] * (self.n - len(prefix) - 1) + prefix
        else:
            prefix = prefix[-self.n + 1:]
        possible_ends = self.ngrams[tuple(prefix)]
        sum_freq = np.log(sum(possible_ends[e] for e in possible_ends))
        return {e: np.log(possible_ends[e]) - sum_freq for e in possible_ends}
    
    def sample(self, prefix):
        possible_ends = self.get_log_probs(prefix)
        if len(possible_ends) > 0:
            end = np.random.choice(list(possible_ends.keys()), p=np.exp(list(possible_ends.values())))
            return end
        return EOS

In [12]:
n_gram_model = NGramModel(n=5)

In [13]:
n_gram_model.compute_ngrams(dataset["train"])

  0%|          | 0/30000 [00:00<?, ?it/s]

In [14]:
idx2token = {idx: token for token, idx in token2idx.items()}

In [15]:
prefix = ["def", "train", "("]
encoded_prefix = [token2idx[token] for token in prefix]
length=100

for i in range(length):
    cur_token = n_gram_model.sample(encoded_prefix)
    if cur_token == EOS:
        break
    encoded_prefix += [cur_token]


decoded_text = [idx2token[idx] for idx in encoded_prefix]
print(" ".join(decoded_text))

def train ( self , data ) : jmodel = sc . _jvm . functions . slice ( _to_java_column ( x ) , - 1 , beam_size , 0 ) ) trans_b = int ( old_attr . get ( 'alpha' , 1.0 ) ) gates_without_top_1 = raw_gates * ( 1.0 - mask_1 ) [UNK] index_2 , gate_2 = mtf . [UNK] ( hparams ) return hparams


In [16]:
test_dataset = load_dataset(
    "json",
    data_files=[
        "test.jsonl",
    ],
)

Using custom data configuration default-0036b8b16813312a


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0036b8b16813312a/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
max_seq_len=128

test_dataset = test_dataset.map(
    lambda item: {
        "features": [encode(token) for token in item["code_tokens"]][:max_seq_len-1] + [EOS]
    }
)

  0%|          | 0/22176 [00:00<?, ?ex/s]

### Метрика качества

$$
P P(p):=2^{H(p)}=2^{-\sum_{x} p(x) \log _{2} p(x)}
$$

Можно использовать

$$
P P' (p):=e^{H(p)}=e^{-\sum_{x} p(x) \ln p(x)}
$$

$$
P P' (p):=e^{H(p)}=e^{-\frac{1}{n}\sum_{x} p(x) \ln p(x)}
$$

In [18]:
def count_perplexity(model, dataset, max_iter_num: int = 1000):
    entropy = 0
    iter_num = 0
    num_words = 0
    for item in tqdm(dataset, total=min(max_iter_num, len(dataset))):
        output_so_far = [item["features"][0]]

        for token in item["features"][1:]:
            num_words += 1
            try:
                log_probs = model.get_log_probs(output_so_far)
                entropy += -log_probs[token]
            except KeyError:
                entropy += np.log(-10)
            output_so_far.append(token)
        iter_num += 1
        if iter_num > max_iter_num:
            break
    mean_entropy = entropy / num_words
    return np.e ** mean_entropy

In [19]:
count_perplexity(n_gram_model, test_dataset["train"])

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in log


nan

## CNN

![](https://lena-voita.github.io/resources/lectures/lang_models/neural/cnn/cnn_main-min.png)



In [20]:
dataset.set_format(type="torch", columns=["features"])
test_dataset.set_format(type="torch", columns=["features"])

In [21]:
def collate_fn(batch):
    batch = batch[0]
    max_len = max(len(f_t) for f_t in batch["features"])
    input_embeds = torch.zeros((len(batch["features"]), max_len), dtype=torch.long)
    for idx, row in enumerate(batch["features"]):
        input_embeds[idx][:len(row)] += row
    return {
        "features": input_embeds,
    }

In [22]:
from torch.utils.data import Sampler


class TextSampler(Sampler):
    def __init__(self, sampler, batch_size_tokens=1e4):
        self.sampler = sampler
        self.batch_size_tokens = batch_size_tokens

    def __iter__(self):
        batch = []
        max_len = 0
        for ix in self.sampler:
            row = self.sampler.data_source[ix]
            max_len = max(max_len, len(row["features"]))
            if (len(batch) + 1) * max_len > self.batch_size_tokens:
                yield batch
                batch = []
                max_len = len(row["features"])
            batch.append(ix)
        if len(batch) > 0:
            yield batch

    def __len__(self):
        return len(self.sampler)

In [23]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler, random_split


train_sampler = RandomSampler(dataset["train"])
valid_sampler = SequentialSampler(test_dataset["train"])

loaders = {
    "train": DataLoader(
        dataset["train"], 
        collate_fn=collate_fn, 
        sampler=TextSampler(train_sampler,)
    ),
    "valid": DataLoader(
        test_dataset["train"],
        collate_fn=collate_fn, 
        sampler=TextSampler(
            valid_sampler, 
        )
    )
}

In [24]:
import torch
import torch.nn as nn


class CNNLM(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, num_layers=3, kernel_size: int = 5):
        super().__init__()
        
        self.emb = nn.Embedding(vocab_size, emb_size)
        layers = []
        for layer_idx in range(num_layers):
            layers.append(nn.ZeroPad2d((kernel_size-1, 0, 0, 0)))
            if layer_idx == 0:
                layers.append(nn.Conv1d(emb_size, hidden_size, kernel_size=kernel_size))
            else:
                layers.append(nn.Conv1d(hidden_size, hidden_size, kernel_size=kernel_size))
        self.conv_layers = nn.Sequential(*layers)
        self.receptive_field = kernel_size + (kernel_size-1)*(num_layers-1)
        self.pred = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_ids):
        embed = self.emb(input_ids)
        embed = embed.permute(0, 2, 1)
        features = self.conv_layers(embed)
        features = features.permute(0, 2, 1)
        logits = self.pred(features)
        return logits

In [25]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = CNNLM(len(tokens) + 3, 300, 100, num_layers=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss(ignore_index=PAD)

In [26]:
from tqdm.notebook import tqdm, trange


def train(
    num_epochs: int, 
    model: nn.Module,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    max_grad_norm: float = None
):
    for epoch in trange(num_epochs):
        pbar = tqdm(train_loader, leave=False, total=len(train_loader)//20)
        pbar.set_description("Train epoch")

        model.train()
        for batch in pbar:

            optimizer.zero_grad()
            features = batch["features"].to(device)
            predictions = model(features[:, :-1])
            loss = criterion(
                predictions.reshape(-1, predictions.size(-1)),
                features[:, 1:].reshape(-1)
            )
            loss.backward()

            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            
        model.eval()
        mean_loss = 0
        pbar = tqdm(valid_loader, leave=False, total=len(valid_loader)//100)
        pbar.set_description("Valid epoch")
        num_iter=0
        for batch in pbar:
            features = batch["features"].to(device)
            with torch.no_grad():
                predictions = model(features[:, :-1])
                loss = criterion(
                    predictions.reshape(-1, predictions.size(-1)),
                    features[:, 1:].reshape(-1)
                )
            mean_loss += loss.item()
            num_iter += 1
        mean_loss /= num_iter
        print(f"Epoch: {epoch}; mean loss: {mean_loss}; perplexity: {np.exp(mean_loss)}")
            

In [27]:
train(
    num_epochs=1,
    model=model, 
    train_loader=loaders["train"],
    valid_loader=loaders["valid"],
    criterion=criterion,
    optimizer=optimizer,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/221 [00:00<?, ?it/s]

Epoch: 0; mean loss: 3.4073929560811895; perplexity: 30.186444200640842


![](https://lena-voita.github.io/resources/lectures/lang_models/neural/cnn/receptive_field-min.png)


Как увеличить receptive field? 

Добавить больше слоев.

Как обучать?

Добавить residual connections.


![](https://lena-voita.github.io/resources/lectures/lang_models/neural/cnn/cnn_with_residual-min.png)



In [28]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from ipywidgets import interactive
from IPython import display

sns.set(style="whitegrid", font_scale=1.4)

sample = np.random.randn(10)
def plot_temperature(T: float = 1.0):
    plt.figure(figsize=(12, 8))
    plt.title(f"Temperature = {T}")
    probs = np.exp(sample / T) / sum(np.exp(sample / T))
    plt.bar(range(10), probs)
    plt.xlabel("tokens")
    plt.ylabel("probs")
    plt.show()


v = interactive(
    plot_temperature, T=(0.02, 10)
)

In [29]:
display.display(v)

interactive(children=(FloatSlider(value=1.0, description='T', max=10.0, min=0.02), Output()), _dom_classes=('w…

In [30]:
from typing import List
from torch.distributions import Categorical

@torch.no_grad()
def generate(
    prefix, model, length: int = 100, receptive_field: int = 5, T: float = 1.
) -> List[int]:
    prefix = torch.from_numpy(prefix)
    prefix = prefix.unsqueeze(0).to(device)
    model.eval()
    for iter_idx in range(length):
        preds = model(prefix[:, -receptive_field:])
        probs = torch.softmax(preds[:, -1]/T, dim=-1)
        distribution = Categorical(probs)
        sampled = distribution.sample()
        if sampled.item() == EOS:
            break
        prefix = torch.cat((prefix, sampled.unsqueeze(0)), dim=1)
    return prefix

In [31]:
prefix = ["def", "train", "("]
encoded_prefix = np.array([token2idx[t] for t in prefix])


for t in np.logspace(0.002, 1, 10):
    generated = generate(
        encoded_prefix, 
        model, 
        receptive_field=model.receptive_field, 
        length=20,
        T=t-1
    )
    print(f"Temperature: {t-1}")
    print(" ".join([idx2token[idx] for idx in generated.cpu().numpy().flatten()]))

Temperature: 0.004615790278395204
def train ( x , train_file , y , [UNK] ) : if self . [UNK] : [UNK] = [UNK] ( [UNK] ,
Temperature: 0.29684743947274783
def train ( x , train_file , y , [UNK] , [UNK] , [UNK] , [UNK] , [UNK] , [UNK] ) : [UNK]
Temperature: 0.6740860511469413
def train ( cls , train_file , _outgoing_pending_queues : datacenter = [UNK] [ mx . sym . currentBlock ( ) . previous )
Temperature: 1.161059212781561
def train ( sess , orientation ) nan ) mask . axis = self . _get_level_number ( image ) mx_uint = 'alias' filter_shape
Temperature: 1.7896874942291365
def train ( __opts__ = state , _expand_item try : resilient_parsing = BotoServerError avail parse_args installer . 'ret_config' ) cell_text bar : ServerError
Temperature: 2.6011768069240193
def train ( shape conv2_2 CommandNotFoundError IDENTITY is best_valid_bleu import 'globals' label 'granularity' filter_select_enabled ) group_list _hack_namedtuple old tags . pool_properties != share_class_symbol
Temperature: 3.6487194

## LSTM

![](https://lena-voita.github.io/resources/lectures/lang_models/neural/rnn/rnn_simple-min.png)

In [32]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size, batch_first=True)
        self.pred = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_ids):
        embs = self.emb(input_ids)
        output, _ = self.lstm(embs)
        return self.pred(output)

In [33]:
model = LSTM(len(token2idx), 300, 50).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [34]:
train(
    num_epochs=1,
    model=model,
    train_loader=loaders["train"],
    valid_loader=loaders["valid"],
    criterion=criterion,
    optimizer=optimizer,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/221 [00:00<?, ?it/s]

Epoch: 0; mean loss: 3.186763223848845; perplexity: 24.209938334760363


## Методы генерации текста

### Greedy Search

$$
w_t = \operatorname{argmax}_{w} P\left(w \mid w_{1: t-1}\right)
$$

![](https://huggingface.co/blog/assets/02_how-to-generate/greedy_search.png)

**Проблема**: Модель быстро начинает повторять одну и ту же фразу. 

### Beam search

![](https://huggingface.co/blog/assets/02_how-to-generate/beam_search.png)

**Проблема**: Модель все еще выдает слишком предсказуемый текст, в отличии от человеческой речи.
![](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)

### Sampling

$$
w_{t} \sim P\left(w \mid w_{1: t-1}\right)
$$

![](https://huggingface.co/blog/assets/02_how-to-generate/sampling_search_with_temp.png)

**Проблема**: страдает целостность текста. Некоторые фразы получаются слишком случайные.

### Top-K Sampling


![](https://huggingface.co/blog/assets/02_how-to-generate/top_k_sampling.png)

Еще можно использовать top-p sampling. Жадно набирать слова, пока их общая вероятность не станет p.


In [35]:
prefix = ["def", "train", "("]
encoded_prefix = np.array([token2idx[t] for t in prefix])

generated = generate(encoded_prefix, model)

In [36]:
prefix = ["def", "train", "("]
encoded_prefix = np.array([token2idx[t] for t in prefix])


for t in np.logspace(0.002, 1, 10):
    generated = generate(
        encoded_prefix, 
        model, 
        receptive_field=20, 
        length=20,
        T=t-1
    )
    print(f"Temperature: {t-1}")
    print(" ".join([idx2token[idx] for idx in generated.cpu().numpy().flatten()]))

Temperature: 0.004615790278395204
def train ( self , [UNK] ) : if not isinstance ( [UNK] , [UNK] ) : if not isinstance ( [UNK] ,
Temperature: 0.29684743947274783
def train ( self , [UNK] , value ) : self . [UNK] ( self . [UNK] ) [UNK] [UNK] [UNK] = [UNK]
Temperature: 0.6740860511469413
def train ( self ) : self . [UNK] ( self . [UNK] ) except [UNK] : for item in range ( [UNK]
Temperature: 1.161059212781561
def train ( self , dtype = self , [UNK] = "Editor" , password = reason [ '.tar.gz' ] = mimedata2url , pandas
Temperature: 1.7896874942291365
def train ( input_tensor regex , filemode , no_unk 'direction' = '{0}/{1}' , force waiter , stdout_stream = mimedata2url permcomps is errors ,
Temperature: 2.6011768069240193
def train ( ps_cmd nmask_train submission_path name success_name alphas zip_ref password texec output modlist decoder_self_attention_bias method_response key_type signal_tiled _pshell no_save_config binaries right_hand_side total_num
Temperature: 3.648719407300855
def train

## References



1.   [Заметки из курса ШАДа.](https://lena-voita.github.io/nlp_course/language_modeling.html)
2.   [Блогпост по теме генерации текста от huggingface.](https://huggingface.co/blog/how-to-generate) Пока не заморачивайтесь, что там за модель в примере. Мы ее подробно рамерем в одном из следующих занятий.

